In [1]:
import os
import time
import datetime
from tensorflow import flags
import tensorflow as tf
import numpy as np
import pandas as pd
import cnn_tool as tool
import random

In [2]:
class TextCNN(object):

    def __init__(
            self, sequence_length, num_classes, vocab_size,
            embedding_size, filter_sizes, num_filters, l2_reg_lambda=0.0):
        # Placeholders for input, output and dropout
        self.input_x = tf.placeholder(tf.int32, [None, sequence_length], name="input_x")
        self.input_y = tf.placeholder(tf.float32, [None, num_classes], name="input_y")
        self.dropout_keep_prob = tf.placeholder(tf.float32, name="dropout_keep_prob")

        # Keeping track of l2 regularization loss (optional)
        l2_loss = tf.constant(0.1)

        # Embedding layer
        with tf.device('/gpu:0'), tf.name_scope("embedding"):
#         with tf.device('/cpu:0'), tf.name_scope("embedding"):
            W = tf.Variable(
                tf.random_uniform([vocab_size, embedding_size], -1.0, 1.0),
                name="W")
            self.embedded_chars = tf.nn.embedding_lookup(W, self.input_x)
            self.embedded_chars_expanded = tf.expand_dims(self.embedded_chars, -1)

        # Create a convolution + maxpool layer for each filter size
        pooled_outputs = []
        for i, filter_size in enumerate(filter_sizes):
            with tf.name_scope("conv-maxpool-%s" % filter_size):
                # Convolution Layer
                filter_shape = [filter_size, embedding_size, 1, num_filters]
                W = tf.Variable(tf.truncated_normal(filter_shape, stddev=0.1), name="W")
                b = tf.Variable(tf.constant(0.1, shape=[num_filters]), name="b")
                conv = tf.nn.conv2d(
                    self.embedded_chars_expanded,
                    W,
                    strides=[1, 1, 1, 1],
                    padding="VALID",
                    name="conv")
                # Apply nonlinearity
                h = tf.nn.relu(tf.nn.bias_add(conv, b), name="relu")
                # Maxpooling over the outputs
                pooled = tf.nn.max_pool(
                    h,
                    ksize=[1, sequence_length - filter_size + 1, 1, 1],
                    strides=[1, 1, 1, 1],
                    padding='VALID',
                    name="pool")
                pooled_outputs.append(pooled)

        # Combine all the pooled features
        num_filters_total = num_filters * len(filter_sizes)
        self.h_pool = tf.concat(pooled_outputs, 3)  # flip params                                                  
#         self.h_pool = tf.concat(3, pooled_outputs)
        self.h_pool_flat = tf.reshape(self.h_pool, [-1, num_filters_total])

        # Add dropout
        with tf.name_scope("dropout"):
            self.h_drop = tf.nn.dropout(self.h_pool_flat, self.dropout_keep_prob)

        # Final (unnormalized) scores and predictions
        with tf.name_scope("output"):
            W = tf.get_variable(
                "W",
                shape=[num_filters_total, num_classes],
                initializer=tf.contrib.layers.xavier_initializer())
            b = tf.Variable(tf.constant(0.1, shape=[num_classes]), name="b")
            l2_loss += tf.nn.l2_loss(W)
            l2_loss += tf.nn.l2_loss(b)
            self.scores = tf.nn.xw_plus_b(self.h_drop, W, b, name="scores")
            self.predictions = tf.argmax(self.scores, 1, name="predictions")

        # Calculate Mean cross-entropy loss
        with tf.name_scope("loss"):
            losses = tf.nn.softmax_cross_entropy_with_logits(labels = self.input_y, logits = self.scores) #  only named arguments accepted                                         
#             losses = tf.nn.softmax_cross_entropy_with_logits(self.scores, self.input_y)
            self.loss = tf.reduce_mean(losses) + l2_reg_lambda * l2_loss

        # Accuracy
        with tf.name_scope("accuracy"):
            correct_predictions = tf.equal(self.predictions, tf.argmax(self.input_y, 1))
            self.accuracy = tf.reduce_mean(tf.cast(correct_predictions, "float"), name="accuracy")

In [3]:
cnn = pd.read_csv("news_dataset.csv", index_col = 0)
contents = cnn["content"]
labels = cnn["label"]
max_document_length = 1000 # max_documemt_length : 2035

# trandform document to vector
x, vocabulary, vocab_size = tool.make_input(contents, max_document_length)
y = tool.make_output(labels)
x_train, x_test, y_train, y_test = tool.divide(x, y, train_prop = 0.8)

In [4]:
# Model Hyperparameters
flags.DEFINE_integer("embedding_dim", 128, "Dimensionality of embedded vector (default: 128)")
flags.DEFINE_string("filter_sizes", "3,4,5", "Comma-separated filter sizes (default: '3,4,5')")
flags.DEFINE_integer("num_filters", 128, "Number of filters per filter size (default: 128)")
flags.DEFINE_float("dropout_keep_prob", 0.4, "Dropout keep probability (default: 0.5)")
flags.DEFINE_float("l2_reg_lambda", 0.1, "L2 regularization lambda (default: 0.0)")

# Training parameters
flags.DEFINE_integer("batch_size", 64, "Batch Size (default: 64)")
flags.DEFINE_integer("num_epochs", 100, "Number of training epochs (default: 200)")
flags.DEFINE_integer("evaluate_every", 100, "Evaluate model on dev set after this many steps (default: 100)")
flags.DEFINE_integer("checkpoint_every", 100, "Save model after this many steps (default: 100)")
flags.DEFINE_integer("num_checkpoints", 5, "Number of checkpoints to store (default: 5)")

# Misc Parameters
flags.DEFINE_boolean("allow_soft_placement", True, "Allow device soft device placement")
flags.DEFINE_boolean("log_device_placement", False, "Log placement of ops on devices")

flags.DEFINE_string('f', '', 'kernel')

FLAGS = tf.flags.FLAGS

In [5]:
# 3. train the model and test
with tf.Graph().as_default():
    sess = tf.Session()
    with sess.as_default():
        cnn = TextCNN(sequence_length=x_train.shape[1],
                      num_classes=y_train.shape[1],
                      vocab_size=vocab_size,
                      embedding_size=FLAGS.embedding_dim,
                      filter_sizes=list(map(int, FLAGS.filter_sizes.split(","))),
                      num_filters=FLAGS.num_filters,
                      l2_reg_lambda=FLAGS.l2_reg_lambda)

        # Define Training procedure
        global_step = tf.Variable(0, name="global_step", trainable=False)
        optimizer = tf.train.AdamOptimizer(1e-3)
        grads_and_vars = optimizer.compute_gradients(cnn.loss)
        train_op = optimizer.apply_gradients(grads_and_vars, global_step=global_step)

        # Keep track of gradient values and sparsity (optional)
        grad_summaries = []
        for g, v in grads_and_vars:
            if g is not None:
                grad_hist_summary = tf.summary.histogram("{}".format(v.name), g)
                sparsity_summary = tf.summary.scalar("{}".format(v.name), tf.nn.zero_fraction(g))
                grad_summaries.append(grad_hist_summary)
                grad_summaries.append(sparsity_summary)
        grad_summaries_merged = tf.summary.merge(grad_summaries)

        # Output directory for models and summaries
        timestamp = str(int(time.time()))
        out_dir = os.path.abspath(os.path.join(os.path.curdir, "runs", timestamp))
        print("Writing to {}\n".format(out_dir))

        # Summaries for loss and accuracy
        loss_summary = tf.summary.scalar("loss", cnn.loss)
        acc_summary = tf.summary.scalar("accuracy", cnn.accuracy)

        # Train Summaries
        train_summary_op = tf.summary.merge([loss_summary, acc_summary, grad_summaries_merged])
        train_summary_dir = os.path.join(out_dir, "summaries", "train")
        train_summary_writer = tf.summary.FileWriter(train_summary_dir, sess.graph)

        # Dev summaries
        dev_summary_op = tf.summary.merge([loss_summary, acc_summary])
        dev_summary_dir = os.path.join(out_dir, "summaries", "dev")
        dev_summary_writer = tf.summary.FileWriter(dev_summary_dir, sess.graph)

        # Checkpoint directory. Tensorflow assumes this directory already exists so we need to create it
        checkpoint_dir = os.path.abspath(os.path.join(out_dir, "checkpoints"))
        checkpoint_prefix = os.path.join(checkpoint_dir, "model")
        if not os.path.exists(checkpoint_dir):
            os.makedirs(checkpoint_dir)
        saver = tf.train.Saver(tf.global_variables(), max_to_keep=FLAGS.num_checkpoints)

        # Initialize all variables
        sess.run(tf.global_variables_initializer())

        def train_step(x_batch, y_batch):
            """
            A single training step
            """
            feed_dict = {
                cnn.input_x: x_batch,
                cnn.input_y: y_batch,
                cnn.dropout_keep_prob: FLAGS.dropout_keep_prob
            }
            _, step, summaries, loss, accuracy = sess.run(
                [train_op, global_step, train_summary_op, cnn.loss, cnn.accuracy],
                feed_dict)
            time_str = datetime.datetime.now().isoformat()
            print("{}: step {}, loss {:g}, acc {:g}".format(time_str, step, loss, accuracy))
            train_summary_writer.add_summary(summaries, step)


        def dev_step(x_batch, y_batch, writer=None):
            """
            Evaluates model on a dev set
            """
            feed_dict = {
                cnn.input_x: x_batch,
                cnn.input_y: y_batch,
                cnn.dropout_keep_prob: 1.0
            }
            step, summaries, loss, accuracy = sess.run(
                [global_step, dev_summary_op, cnn.loss, cnn.accuracy],
                feed_dict)
            time_str = datetime.datetime.now().isoformat()
            print("{}: step {}, loss {:g}, acc {:g}".format(time_str, step, loss, accuracy))
            if writer:
                writer.add_summary(summaries, step)


        def batch_iter(data, batch_size, num_epochs, shuffle=True):
            """
            Generates a batch iterator for a dataset.
            """
            data = np.array(data)
            data_size = len(data)
            num_batches_per_epoch = int((len(data) - 1) / batch_size) + 1
            for epoch in range(num_epochs):
                # Shuffle the data at each epoch
                if shuffle:
                    shuffle_indices = np.random.permutation(np.arange(data_size))
                    shuffled_data = data[shuffle_indices]
                else:
                    shuffled_data = data
                for batch_num in range(num_batches_per_epoch):
                    start_index = batch_num * batch_size
                    end_index = min((batch_num + 1) * batch_size, data_size)
                    yield shuffled_data[start_index:end_index]


        # Generate batches
        batches = batch_iter(
            list(zip(x_train, y_train)), FLAGS.batch_size, FLAGS.num_epochs)

        # Training loop. For each batch...
        print(global_step)
        for batch in batches:
            x_batch, y_batch = zip(*batch)
            train_step(x_batch, y_batch)
            current_step = tf.train.global_step(sess, global_step)
            if current_step % FLAGS.evaluate_every == 0:
                print("\nEvaluation:")
                dev_step(x_test, y_test, writer=dev_summary_writer)
                print("")
            if current_step % FLAGS.checkpoint_every == 0:
                path = saver.save(sess, checkpoint_prefix, global_step=current_step)
                print("Saved model checkpoint to {}\n".format(path))                

INFO:tensorflow:Summary name embedding/W:0 is illegal; using embedding/W_0 instead.
INFO:tensorflow:Summary name embedding/W:0 is illegal; using embedding/W_0 instead.
INFO:tensorflow:Summary name conv-maxpool-3/W:0 is illegal; using conv-maxpool-3/W_0 instead.
INFO:tensorflow:Summary name conv-maxpool-3/W:0 is illegal; using conv-maxpool-3/W_0 instead.
INFO:tensorflow:Summary name conv-maxpool-3/b:0 is illegal; using conv-maxpool-3/b_0 instead.
INFO:tensorflow:Summary name conv-maxpool-3/b:0 is illegal; using conv-maxpool-3/b_0 instead.
INFO:tensorflow:Summary name conv-maxpool-4/W:0 is illegal; using conv-maxpool-4/W_0 instead.
INFO:tensorflow:Summary name conv-maxpool-4/W:0 is illegal; using conv-maxpool-4/W_0 instead.
INFO:tensorflow:Summary name conv-maxpool-4/b:0 is illegal; using conv-maxpool-4/b_0 instead.
INFO:tensorflow:Summary name conv-maxpool-4/b:0 is illegal; using conv-maxpool-4/b_0 instead.
INFO:tensorflow:Summary name conv-maxpool-5/W:0 is illegal; using conv-maxpool-5

2018-06-22T16:34:43.236053: step 101, loss 2.09526, acc 0.703125
2018-06-22T16:34:43.457742: step 102, loss 1.43208, acc 0.734375
2018-06-22T16:34:43.661221: step 103, loss 1.95638, acc 0.703125
2018-06-22T16:34:43.883950: step 104, loss 1.42695, acc 0.734375
2018-06-22T16:34:44.045326: step 105, loss 1.95079, acc 0.694444
2018-06-22T16:34:44.251069: step 106, loss 1.90465, acc 0.671875
2018-06-22T16:34:44.486617: step 107, loss 1.73467, acc 0.734375
2018-06-22T16:34:44.688304: step 108, loss 1.52785, acc 0.734375
2018-06-22T16:34:44.918206: step 109, loss 2.16141, acc 0.59375
2018-06-22T16:34:45.114229: step 110, loss 1.0143, acc 0.833333
2018-06-22T16:34:45.341105: step 111, loss 1.81772, acc 0.6875
2018-06-22T16:34:45.576030: step 112, loss 1.28056, acc 0.765625
2018-06-22T16:34:45.783566: step 113, loss 2.01795, acc 0.703125
2018-06-22T16:34:45.982512: step 114, loss 1.46467, acc 0.75
2018-06-22T16:34:46.151170: step 115, loss 1.21495, acc 0.805556
2018-06-22T16:34:46.381958: step 

2018-06-22T16:35:08.935581: step 226, loss 0.854334, acc 0.875
2018-06-22T16:35:09.153269: step 227, loss 0.851761, acc 0.84375
2018-06-22T16:35:09.391824: step 228, loss 0.996407, acc 0.78125
2018-06-22T16:35:09.610835: step 229, loss 0.57113, acc 0.953125
2018-06-22T16:35:09.802841: step 230, loss 0.60051, acc 0.916667
2018-06-22T16:35:10.011539: step 231, loss 1.07028, acc 0.890625
2018-06-22T16:35:10.220634: step 232, loss 0.756386, acc 0.890625
2018-06-22T16:35:10.417558: step 233, loss 0.729369, acc 0.875
2018-06-22T16:35:10.639235: step 234, loss 0.838809, acc 0.890625
2018-06-22T16:35:10.819815: step 235, loss 0.972569, acc 0.805556
2018-06-22T16:35:11.027370: step 236, loss 1.12562, acc 0.859375
2018-06-22T16:35:11.244603: step 237, loss 1.0938, acc 0.8125
2018-06-22T16:35:11.448612: step 238, loss 0.66319, acc 0.84375
2018-06-22T16:35:11.650632: step 239, loss 0.628021, acc 0.890625
2018-06-22T16:35:11.831353: step 240, loss 1.08617, acc 0.833333
2018-06-22T16:35:12.024509: s

2018-06-22T16:35:34.033038: step 351, loss 0.732546, acc 0.890625
2018-06-22T16:35:34.261395: step 352, loss 0.492554, acc 0.9375
2018-06-22T16:35:34.485103: step 353, loss 0.56962, acc 0.921875
2018-06-22T16:35:34.677718: step 354, loss 1.10195, acc 0.875
2018-06-22T16:35:34.850293: step 355, loss 0.491303, acc 0.944444
2018-06-22T16:35:35.029206: step 356, loss 0.464487, acc 0.921875
2018-06-22T16:35:35.223391: step 357, loss 0.616214, acc 0.9375
2018-06-22T16:35:35.423760: step 358, loss 0.668448, acc 0.9375
2018-06-22T16:35:35.632458: step 359, loss 0.5592, acc 0.90625
2018-06-22T16:35:35.817449: step 360, loss 0.70665, acc 0.861111
2018-06-22T16:35:36.016816: step 361, loss 0.47868, acc 0.9375
2018-06-22T16:35:36.225113: step 362, loss 0.496205, acc 0.9375
2018-06-22T16:35:36.421900: step 363, loss 0.480303, acc 0.9375
2018-06-22T16:35:36.628605: step 364, loss 0.473332, acc 0.9375
2018-06-22T16:35:36.808958: step 365, loss 0.644039, acc 0.916667
2018-06-22T16:35:37.017664: step 3

2018-06-22T16:35:59.640820: step 474, loss 0.598835, acc 0.875
2018-06-22T16:35:59.835397: step 475, loss 0.496668, acc 0.944444
2018-06-22T16:36:00.060749: step 476, loss 0.381641, acc 0.953125
2018-06-22T16:36:00.247351: step 477, loss 0.357865, acc 0.984375
2018-06-22T16:36:00.456610: step 478, loss 0.437296, acc 0.953125
2018-06-22T16:36:00.680465: step 479, loss 0.503902, acc 0.921875
2018-06-22T16:36:00.846186: step 480, loss 0.379788, acc 0.944444
2018-06-22T16:36:01.039185: step 481, loss 0.325049, acc 0.984375
2018-06-22T16:36:01.239352: step 482, loss 0.377515, acc 0.984375
2018-06-22T16:36:01.474788: step 483, loss 0.304569, acc 1
2018-06-22T16:36:01.679675: step 484, loss 0.389296, acc 0.96875
2018-06-22T16:36:01.862650: step 485, loss 0.517708, acc 0.944444
2018-06-22T16:36:02.048119: step 486, loss 0.387618, acc 0.953125
2018-06-22T16:36:02.227969: step 487, loss 0.337502, acc 0.984375
2018-06-22T16:36:02.459904: step 488, loss 0.357688, acc 0.96875
2018-06-22T16:36:02.65

텐서플로우를 이용하여 text_cnn classification을 수행한 결과 train dataset에 대해서는 accuracy가 거의 1에 가까울 정도로 우수하였으나,
test data에 대한 val_acc는 0.63 정도에 불과하여 모델이 과적합이 발생한 것으로 사료됨.